<h1>CS152 Assignment 1: The 8-puzzle</h1>

Before you turn in this assignment, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then run the test cells for each of the questions you have answered.  Note that a grade of 3 for the A* implementation requires all tests in the "Basic Functionality" section to be passed.  The test cells pass if they execute with no errors (i.e. all the assertions are passed).

Make sure you fill in any place that says `YOUR CODE HERE`.  Be sure to remove the `raise NotImplementedError()` statements as you implement your code - these are simply there as a reminder if you forget to add code where it's needed.

---

<h1>
Question 1    
</h1>
Define your <code>PuzzleNode</code> class below.  Ensure that you include all attributes that you need to implement an A* search.  If you wish, you can even include member functions, such as a function to generate successor states.  Alternatively, you can code up this functionality later in the <code>solvePuzzle</code> function.

In [4]:
# Import any packages you need here
# Also define any variables as needed
import copy

#Now, define the class PuzzleNode:
class PuzzleNode:
    """
    Class PuzzleNode: Provides a structure for performing A* search for the n^2-1 puzzle
    """
    
    # initialization
    def __init__(self, state, gval=0, fval=0, parent=None):
        if len(state) == 0 or type(state[0]) != list:
            raise ValueError("This is a 2 dimensional world. Comply.")
        
        # I could also check values but I'm lazy
            
        self.state = state
        self.parent = parent
         
        self.G = gval # dist from start node
        self.F = fval # g+h
                
        self.dims = (len(state)-1, len(state[0])-1)
    
    # for comparison in priority queue
    def __lt__(self,other):
        return self.F < other.F

    # equality is defined by the same state
    def __eq__(self, other):
        return self.__class__ == other.__class__ and self.state == other.state
    
    # pretty print state
    def __str__(self):
        res = "_____\n"
        for row in self.state:
            res += ' '.join(map(str, row))
            res += '\n'
        res += "_____"
        return res
    
    def next_states(self):
        # find the empty space
        zr = zc = -1
        for i, row in enumerate(self.state):
            try:
                zr, zc = i, row.index(0)
            except ValueError:
                pass
        
        nextStates = []
        
        # make children with one move diff        
        if zr > 0:
            state = copy.deepcopy(self.state)
            state[zr][zc], state[zr-1][zc] = state[zr-1][zc], state[zr][zc]
            nextStates.append(state)
        if zr < self.dims[0]:
            state = copy.deepcopy(self.state)
            state[zr][zc], state[zr+1][zc] = state[zr+1][zc], state[zr][zc]
            nextStates.append(state)
        if zc > 0 :
            state = copy.deepcopy(self.state)
            state[zr][zc], state[zr][zc-1] = state[zr][zc-1], state[zr][zc]
            nextStates.append(state)
        if zc < self.dims[1]:
            state = copy.deepcopy(self.state)
            state[zr][zc], state[zr][zc+1] = state[zr][zc+1], state[zr][zc] 
            nextStates.append(state)
        return(nextStates)
    


<h1>
Question 2    
</h1>
Define your heuristic functions using the templates below.  Ensure that you extend the <code>heuristics</code> list to include all the heuristic functions you implement.  Note that state will be given as a list of lists, so ensure your function accepts this format.  You may use packages like numpy if you wish within the functions themselves.

In [5]:
# Add any additional code here (e.g. for the memoization extension)

# YOUR CODE HERE (OPTIONAL)

# Misplaced tiles heuristic
def h1(state):
    """
    This function returns the number of misplaced tiles, given the board state
    Input:
        -state: the board state as a list of lists
    Output:
        -h: the number of misplaced tiles
    """
    misp = 0
    
    for r, row in enumerate(state):
        for c, t in enumerate(row):
            if t != r*3+c:
                misp += 1
    return(misp)

# Manhattan distance heuristic
def h2(state):
    """
    This function returns the Manhattan distance from the solved state, given the board state
    Input:
        -state: the board state as a list of lists
    Output:
        -h: the Manhattan distance from the solved configuration
    """
    manh = 0
    
    for r, row in enumerate(state):
        for c, t in enumerate(row):
            manh += abs(t//3-r) # row distance
            manh += abs(t%3-c) # col distance
    return(manh)
    
# Extra heuristic for the extension.  If implemented, modify the function below
def h3(state):
    """
    This function returns a heuristic that dominates the Manhattan distance, given the board state
    Input:
        -state: the board state as a list of lists
    Output:
        -h: the Heuristic distance of the state from its solved configuration
    """
    return 0

# If you implement more than 3 heuristics, then add any extra heuristic functions onto the end of this list.
heuristics = [h1, h2, h3]

<h1>
Question 3    
</h1>
Code up your A* search using the SolvePuzzle function within the template below.  Please do not modify the function header, otherwise the automated testing will fail.  You may define other functions or import packages as needed in this cell or by adding additional cells.

In [6]:
# creating child nodes for valid next moves

# Import any packages or define any helper functions you need here

from queue import PriorityQueue

# Main solvePuzzle function.
def solvePuzzle(state, heuristic):
    """This function should solve the n**2-1 puzzle for any n > 2 (although it may take too long for n > 4)).
    Inputs:
        -state: The initial state of the puzzle as a list of lists
        -heuristic: a handle to a heuristic function.  Will be one of those defined in Question 2.
    Outputs:
        -steps: The number of steps to optimally solve the puzzle (excluding the initial state)
        -exp: The number of nodes expanded to reach the solution
        -max_frontier: The maximum size of the frontier over the whole search
        -opt_path: The optimal path as a list of list of lists.  That is, opt_path[:,:,i] should give a list of lists
                    that represents the state of the board at the ith step of the solution.
        -err: An error code.  If state is not of the appropriate size and dimension, return -1.  For the extention task,
          if the state is not solvable, then return -2
    """
    
    # _____ HELPER _____
    def create_goal(state):
        v = 0
        goal = []
        for r, row in enumerate(state):
            arow = []
            for c, _ in enumerate(row):
                arow.append(v)
                v += 1
            goal.append(arow)
        return(goal)
    
    def trace_path(node, path = {}, height = 0):
        if node.parent == None:
            return(path)
        else:
            path[height] = node.state
            return(trace_path(node.parent, path, height + 1))
    
    def immortalize(lst_of_lst):
        return(tuple(tuple(x) for x in lst_of_lst))

    
    # _____ INIT _____
    # create goal state
    goal = create_goal(state)
    
    # create start node
    start_node = PuzzleNode(state)
    start_node.F = heuristic(state)
        
    # priority queue
    frontier = PriorityQueue()
    frontier.put(start_node) # excluding the g(s) = 0
    
    # lookup for best paths found to a state
    clookup = {}
    clookup[immortalize(state)] = start_node
    
    # stats to remember
    expanded = 0
    
    
    closed = []

    
    while not frontier.empty():
        cur_node = frontier.get()
        
        if cur_node.state == goal:
            print("FOUND")
            print(cur_node.state)
            break
        
        for state in cur_node.next_states():
            # cost = dist from start + heuristic
            G = cur_node.G + 1
            H = heuristic(state)
            
            imState = immortalize(state)
            
            if imState not in closed and imState not in frontier:

            if imState in clookup:
                if clookup[imState].G <= G:
                    continue # if better path to this state is known, ignore this path
                else:
                    print("pruned")
                    clookup[imState].pruned = True # else, prune old path
            
            # add new node to frontier, update lookup
            print("ADD: ", state)
            child_node = PuzzleNode(state, G, G + heuristic(state), cur_node)
            frontier.put(child_node)
            clookup[imState] = child_node   
            
            
        expanded += 1
    
    # recreate optimal path
    opath = []
    while cur_node.parent is not None:
        opath.append(cur_node.state)
        cur_node = cur_node.parent
    return(opath)


In [7]:
incorrect_state = [[7,2,4],[5,0,6],[8,3,1]]


In [8]:
print(solvePuzzle(incorrect_state, heuristics[0]))

[[0, 1, 2], [3, 4, 5], [6, 7, 8]]


AttributeError: 'PuzzleNode' object has no attribute 'pruned'

<h1>Extension Questions</h1>

The extensions can be implemented by modifying the code from Q2-3 above appropriately.

1. <b>Initial state solvability:</b>  Modify your SolvePuzzle function code in Q3 to return -2 if an initial state is not solvable to the goal state.
2. <b>Extra heuristic function:</b> Add another heuristic function (e.g. pattern database) that dominates the misplaced tiles and Manhattan distance heuristics to your Q2 code.
3. <b>Memoization:</b>  Modify your heuristic function definitions in Q2 by using a Python decorator to speed up heuristic function evaluation

There are test cells provided for extension questions 1 and 2.

<h1>Basic Functionality Tests</h1>
The cells below contain tests to verify that your code is working properly to be classified as basically functional.  Please note that a grade of <b>3</b> on #aicoding and #search as applicable for each test requires the test to be successfully passed.  <b>If you want to demonstrate some other aspect of your code, then feel free to add additional cells with test code and document what they do.<b>

In [ ]:
## Test for state not correctly defined

incorrect_state = [[0,1,2],[2,3,4],[5,6,7]]
_,_,_,_,err = solvePuzzle(incorrect_state, lambda state: 0)
assert(err == -1)

In [ ]:
## Heuristic function tests for misplaced tiles and manhattan distance

# Define the working initial states
working_initial_states_8_puzzle = ([[2,3,7],[1,8,0],[6,5,4]], [[7,0,8],[4,6,1],[5,3,2]], [[5,7,6],[2,4,3],[8,1,0]])

# Test the values returned by the heuristic functions
h_mt_vals = [7,8,7]
h_man_vals = [15,17,18]

for i in range(0,3):
    h_mt = heuristics[0](working_initial_states_8_puzzle[i])
    h_man = heuristics[1](working_initial_states_8_puzzle[i])
    assert(h_mt == h_mt_vals[i])
    assert(h_man == h_man_vals[i])


In [ ]:
## A* Tests for 3 x 3 boards
## This test runs A* with both heuristics and ensures that the same optimal number of steps are found
## with each heuristic.

# Optimal path to the solution for the first 3 x 3 state
opt_path_soln = [[[2, 3, 7], [1, 8, 0], [6, 5, 4]], [[2, 3, 7], [1, 8, 4], [6, 5, 0]], 
                 [[2, 3, 7], [1, 8, 4], [6, 0, 5]], [[2, 3, 7], [1, 0, 4], [6, 8, 5]], 
                 [[2, 0, 7], [1, 3, 4], [6, 8, 5]], [[0, 2, 7], [1, 3, 4], [6, 8, 5]], 
                 [[1, 2, 7], [0, 3, 4], [6, 8, 5]], [[1, 2, 7], [3, 0, 4], [6, 8, 5]], 
                 [[1, 2, 7], [3, 4, 0], [6, 8, 5]], [[1, 2, 0], [3, 4, 7], [6, 8, 5]], 
                 [[1, 0, 2], [3, 4, 7], [6, 8, 5]], [[1, 4, 2], [3, 0, 7], [6, 8, 5]], 
                 [[1, 4, 2], [3, 7, 0], [6, 8, 5]], [[1, 4, 2], [3, 7, 5], [6, 8, 0]], 
                 [[1, 4, 2], [3, 7, 5], [6, 0, 8]], [[1, 4, 2], [3, 0, 5], [6, 7, 8]], 
                 [[1, 0, 2], [3, 4, 5], [6, 7, 8]], [[0, 1, 2], [3, 4, 5], [6, 7, 8]]]

astar_steps = [17, 25, 28]
for i in range(0,3):
    steps_mt, expansions_mt, _, opt_path_mt, _ = solvePuzzle(working_initial_states_8_puzzle[i], heuristics[0])
    steps_man, expansions_man, _, opt_path_man, _ = solvePuzzle(working_initial_states_8_puzzle[i], heuristics[1])
    # Test whether the number of optimal steps is correct and the same
    assert(steps_mt == steps_man == astar_steps[i])
    # Test whether or not the manhattan distance dominates the misplaced tiles heuristic in every case
    assert(expansions_man < expansions_mt)
    # For the first state, test that the optimal path is the same
    if i == 0:
        assert(opt_path_mt == opt_path_soln)


In [ ]:
## A* Test for 4 x 4 board
## This test runs A* with both heuristics and ensures that the same optimal number of steps are found
## with each heuristic.

working_initial_state_15_puzzle = [[1,2,6,3],[0,9,5,7],[4,13,10,11],[8,12,14,15]]
steps_mt, expansions_mt, _, _, _ = solvePuzzle(working_initial_state_15_puzzle, heuristics[0])
steps_man, expansions_man, _, _, _ = solvePuzzle(working_initial_state_15_puzzle, heuristics[1])
# Test whether the number of optimal steps is correct and the same
assert(steps_mt == steps_man == 9)
# Test whether or not the manhattan distance dominates the misplaced tiles heuristic in every case
assert(expansions_mt >= expansions_man)

<h1>Extension Tests</h1>
The cells below can be used to test the extension questions.  Memoization if implemented will be tested on the final submission - you can test it yourself by testing the execution time of the heuristic functions with and without it.

In [ ]:
## Puzzle solvability test

unsolvable_initial_state = [[7,5,6],[2,4,3],[8,1,0]]
_,_,_,_,err = solvePuzzle(unsolvable_initial_state, lambda state: 0)
assert(err == -2)

In [ ]:
## Extra heuristic function test.  
## This tests that for all initial conditions, the new heuristic dominates over the manhattan distance.

dom = 0
for i in range(0,3):
    steps_new, expansions_new, _, _, _ = solvePuzzle(working_initial_states_8_puzzle[i], heuristics[2])
    steps_man, expansions_man, _, _, _ = solvePuzzle(working_initial_states_8_puzzle[i], heuristics[1])
    # Test whether the number of optimal steps is correct and the same
    assert(steps_new == steps_man == astar_steps[i])
    # Test whether or not the manhattan distance is dominated by the new heuristic in every case, by checking
    # the number of nodes expanded
    dom = expansions_man - expansions_new
assert(dom > 0)

In [ ]:
## Memoization test - will be carried out after submission